# Convert downloaded TCGA datasets into sample × gene matrices

In [1]:
import os

import pandas

## Read sample information

This file contains sample information. See the [online documentation](https://genome-cancer.soe.ucsc.edu/proj/site/xena/datapages/?dataset=TCGA.PANCAN.sampleMap/PANCAN_clinicalMatrix&host=https://tcga.xenahubs.net) for `PANCAN_clinicalMatrix`.

In [2]:
path = os.path.join('download', 'PANCAN_clinicalMatrix.tsv.bz2')
clinmat_df = (
    pandas.read_table(path)
    .rename(columns={'sampleID': 'sample_id'})
)
# Check that no sample_ids are duplicated
assert not clinmat_df.sample_id.duplicated().any()
clinmat_df.shape

(12088, 36)

In [3]:
# Types of samples
clinmat_df.sample_type.value_counts()

Primary Tumor                                      10518
Solid Tissue Normal                                  782
Metastatic                                           396
Primary Blood Derived Cancer - Peripheral Blood      200
Recurrent Tumor                                       46
Additional - New Primary                              11
Additional Metastatic                                  1
Name: sample_type, dtype: int64

## Read mutation data

This file contains mutation data (which mutations each sample contains) See the [online documentation](https://genome-cancer.soe.ucsc.edu/proj/site/xena/datapages/?dataset=TCGA.PANCAN.sampleMap/PANCAN_mutation&host=https://tcga.xenahubs.net) for `PANCAN_mutation`.

In [4]:
path = os.path.join('download', 'PANCAN_mutation.tsv.bz2')
snp_mutation_df = (
    pandas.read_table(path)
    .rename(columns={'sample': 'sample_id'})
)
snp_mutation_df.head(2)

,sample_id,chr,start,end,reference,alt,gene,effect,DNA_VAF,RNA_VAF,Amino_Acid_Change
0,TCGA-D8-A1J8-01,chr10,52587953,52587953,C,A,A1CF,Missense_Mutation,NaN,NaN,p.R236I
1,TCGA-BH-A0HP-01,chr10,52595854,52595854,G,A,A1CF,Missense_Mutation,NaN,NaN,p.A195V


In [5]:
# The mutation data contains duplicated rows. Disturbing issues like this are
# common in bioinformatics. The following
print('Removing {:.2%} of the mutations, which are duplicates'.format(snp_mutation_df.duplicated().sum() / len(snp_mutation_df)))
snp_mutation_df.drop_duplicates(inplace=True)

Removing 4.91% of the mutations, which are duplicates


In [6]:
# Number of samples with at least one mutation
snp_mutation_df.sample_id.nunique()

8024

##### Mutations counts by type

Notice that mutation type codes are not standardized. For example, `5_prime_UTR_variant`, `5'UTR`, `UTR_5_PRIME` all refer to the same type of mutation.

In [7]:
snp_mutation_df.effect.value_counts().reset_index()

,index,effect
0,Missense_Mutation,1044846
1,Silent,432995
2,Nonsense_Mutation,81092
3,RNA,71493
4,Frame_Shift_Del,46941
5,Splice_Site,43262
6,Frame_Shift_Ins,22546
7,missense_variant,20241
8,In_Frame_Del,11455
9,synonymous_variant,7907


### Convert SNP mutations to gene mutations

In [8]:
mutations = {
    'Missense_Mutation',
    'Nonsense_Mutation',
}

In [9]:
gene_mutation_df = (snp_mutation_df
    .query("effect in @mutations")
    .groupby(['sample_id', 'gene'])
    .apply(len)
    .reset_index()
    .rename(columns={0: 'count'})
)
gene_mutation_df.head(2)

,sample_id,gene,count
0,TCGA-02-0003-01,AKAP6,1
1,TCGA-02-0003-01,ANAPC4,1


In [10]:
# Create a sample (rows) by gene (columns) matrix of mutation status
gene_mutation_mat_df = (gene_mutation_df
    .pivot_table(index='sample_id', columns='gene', values='count', fill_value=0)
    .astype(bool).astype(int)
)
gene_mutation_mat_df.shape

(7497, 21791)

In [11]:
'{:.2%} sample-gene pairs are mutated'.format(
    gene_mutation_mat_df.stack().mean())

'0.62% sample-gene pairs are mutated'

In [12]:
# Top mutated genes
gene_mutation_df.gene.value_counts().reset_index().head(5)

,index,gene
0,TTN,2210
1,TP53,2045
2,MUC16,1362
3,PIK3CA,969
4,CSMD3,859


In [13]:
# Top mutated samples
gene_mutation_df.sample_id.value_counts().reset_index().head(5)

,index,sample_id
0,TCGA-IB-7651-01,8704
1,TCGA-FW-A3R5-06,7244
2,TCGA-AP-A0LM-01,7024
3,TCGA-AP-A059-01,6426
4,TCGA-B5-A0JY-01,6113


## Read gene expression data

This file contains gene expression data from RNA-Sequencing. See the [online documentation](https://genome-cancer.soe.ucsc.edu/proj/site/xena/datapages/?dataset=TCGA.PANCAN.sampleMap/HiSeqV2&host=https://tcga.xenahubs.net) for `HiSeqV2`.

In [14]:
# Read the gene × sample dataset
path = os.path.join('download', 'HiSeqV2.tsv.bz2')
expr_df = pandas.read_table(path, index_col=0)

# Process the dataset
expr_df = (expr_df
    # Remove genes containing a `?`
    [~expr_df.index.str.contains('?', regex=False)]
    # Sort samples
    .sort_index()
    # Transpose so the data is sample × gene
    .transpose()
    # Sort genes
    .sort_index()
)

expr_df.index.rename('sample_id', inplace=True)

expr_df.shape

(10459, 20501)

In [15]:
# Peak at the data matrix
expr_df.iloc[:5, :5]

Sample,A1BG,A1CF,A2BP1,A2LD1,A2M
sample_id,,,,,
TCGA-02-0047-01,6.9774,0.0,7.9403,5.8092,15.0538
TCGA-02-0055-01,8.6177,0.0,7.1122,6.4096,15.3879
TCGA-02-2483-01,8.0920,0.0,6.8077,5.1513,14.3622
TCGA-02-2485-01,6.4084,0.0,8.0120,6.9919,12.9292
TCGA-02-2486-01,6.7716,0.0,2.3973,7.5814,15.3224


## Integrate expression and mutation data

Find samples with both mutation and expression data. We assume that if a sample was not in `PANCAN_mutation`, it was not assayed for mutation. Hence, zero-mutation cancers are excluded even if they have mutation data.

In [16]:
sample_ids = list(gene_mutation_mat_df.index & expr_df.index)
len(sample_ids)

6825

In [17]:
# Filter expression (x) and mutation (y) matrices for common samples
x_df = expr_df.loc[sample_ids, :]
y_df = gene_mutation_mat_df.loc[sample_ids, :]

### Export matrices to TSVs

Matrices are saved as sample × gene TSVs. Subsetted matrices are also exported to allow users to quickly explore small portions of the dataset.

In [18]:
def sample_df(df, nrows=None, ncols=None, row_seed=0, col_seed=0):
    """Randomly subset a dataframe, preserving row and column order."""
    if nrows is None:
        nrows = len(df)
    if ncols is None:
        ncols = len(df.columns)
    return (df
        .sample(n=nrows, random_state=row_seed, axis='rows')
        .sample(n=ncols, random_state=col_seed, axis='columns')
        .sort_index(axis='rows')
        .sort_index(axis='columns')
    )

In [19]:
tsv_args = {'sep': '\t', 'float_format': '%.3g'}

for df, name in (x_df, 'expression-matrix'), (y_df, 'mutation-matrix'):

    # Save full dataset
    path = os.path.join('data', name + '.tsv.bz2')
    df.to_csv(path, **tsv_args, compression='bz2')
    
    # Save subsetted datasets
    for sample, nrows, ncols in ('small', 50, 15), ('all-samples', None, 15), ('all-genes', 50, None):
        path = os.path.join('data', 'subset', '{}-{}.tsv'.format(name, sample))
        sample_df(df, nrows=nrows, ncols=ncols).to_csv(path, **tsv_args)